### Data Exploration 2 Drill

In this assignment, you keep working with the Students Performance in Exams dataset from Kaggle.

To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

1) Are there any differences between the genders, ethnicities, and parental level of education with respect to their performances in exams?

2) Are there any differences between the lunch types with respect to their performances in exams? If there are, how do you explain this?

3) Does the test preparation course seem to have an effect on the exam performances?

4) Which 2 exam scores are most correlated with each other?

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")


In [2]:
#Import dataset from postgres sql server

#defines the logon credentials
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'studentsperformance'

#opens the connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host,
    postgres_port, postgres_db))

#queries the schema
student_df = pd.read_sql_query('select * from studentsperformance', con=engine)

#closes the connection
engine.dispose()


### 1) Check the data & clean if necessary

student_df.info()

In [3]:
np.round(student_df.describe(include='all'),2)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
count,1000,1000,1000,1000,1000,1000.00,1000.00,1000.00
unique,2,5,6,2,2,NaN,NaN,NaN
top,female,group C,some college,standard,none,NaN,NaN,NaN
freq,518,319,226,645,642,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,66.09,69.17,68.05
std,NaN,NaN,NaN,NaN,NaN,15.16,14.60,15.20
min,NaN,NaN,NaN,NaN,NaN,0.00,17.00,10.00
25%,NaN,NaN,NaN,NaN,NaN,57.00,59.00,57.75
50%,NaN,NaN,NaN,NaN,NaN,66.00,70.00,69.00
75%,NaN,NaN,NaN,NaN,NaN,77.00,79.00,79.00


### 2) Are there any differences between the genders, ethnicities, and parental level of education with respect to their performances in exams?

In [4]:
#Groupby & visualize the data to spot differences

#Groupby gender
gender_df=student_df.groupby('gender').mean()[['math score', 'writing score', 'reading score']]
gender_df

,math score,writing score,reading score
gender,,,
female,63.633205,72.467181,72.608108
male,68.728216,63.311203,65.473029


In [5]:
#Perform t test to determine if differences in score are significant

stats.ttest_ind(student_df[student_df.gender =='female'][['math score', 'writing score', 'reading score']],
                student_df[student_df.gender =='male'][['math score', 'writing score', 'reading score']]
)

Ttest_indResult(statistic=array([-5.38324587,  9.97955791,  7.95930801]), pvalue=array([9.12018555e-08, 2.01987771e-22, 4.68053874e-15]))

Female students appear to be better at writing and reading than male students.  The students' scores vary significantly between the genders especially in writing and reading. 

In [6]:
#Groupby ethnicity
ethnicity=student_df.groupby(['race/ethnicity']).mean()
ethnicity

,math score,reading score,writing score
race/ethnicity,,,
group A,61.629213,64.674157,62.674157
group B,63.452632,67.352632,65.600000
group C,64.463950,69.103448,67.827586
group D,67.362595,70.030534,70.145038
group E,73.821429,73.028571,71.407143


In [7]:
#Perform t test to determine if differences in score are significant

ethnicity = student_df["race/ethnicity"].unique()
grouped_df = student_df.groupby("race/ethnicity")
for var in ["math score", "reading score", "writing score"]:
    print("------------------------------------------------")
    print("Comparisons for variable: {}".format(var))
    print("------------------------------------------------")
    for i in range(0, len(ethnicity)):
        for j in range(i+1, len(ethnicity)):
            print("t-test between groups {0} and {1}:".format(ethnicity[i], ethnicity[j]))
            print(stats.ttest_ind(student_df[student_df["race/ethnicity"]== ethnicity[i]][var],
                student_df[student_df["race/ethnicity"] == ethnicity[j]][var]))

------------------------------------------------
Comparisons for variable: math score
------------------------------------------------
t-test between groups group B and group C:
Ttest_indResult(statistic=-0.7315669893534263, pvalue=0.4647708939167453)
t-test between groups group B and group A:
Ttest_indResult(statistic=0.9355076279747488, pvalue=0.3503415961659957)
t-test between groups group B and group D:
Ttest_indResult(statistic=-2.82845539712675, pvalue=0.004886197137104194)
t-test between groups group B and group E:
Ttest_indResult(statistic=-6.007397050552227, pvalue=5.007946047497971e-09)
t-test between groups group C and group A:
Ttest_indResult(statistic=1.5997220303217299, pvalue=0.11043810745588042)
t-test between groups group C and group D:
Ttest_indResult(statistic=-2.41858624746011, pvalue=0.015888349556016285)
t-test between groups group C and group E:
Ttest_indResult(statistic=-6.127642520822135, pvalue=1.927238849552764e-09)
t-test between groups group A and group D:


Group E has the highest average scores among the 5 groups, followed by Group D.  Group A has the lowest average scores.  The t-tests show that the differences between Groups E & D and Group A are significant while the differences between Group E & D are not significant.

In [8]:
#Groupby parent education
parent=student_df.groupby(['parental level of education']).mean()
parent

,math score,reading score,writing score
parental level of education,,,
associate's degree,67.882883,70.927928,69.896396
bachelor's degree,69.389831,73.000000,73.381356
high school,62.137755,64.704082,62.448980
master's degree,69.745763,75.372881,75.677966
some college,67.128319,69.460177,68.840708
some high school,63.497207,66.938547,64.888268


In [9]:
#Perform t test to determine if differences in score are significant

parent = student_df["parental level of education"].unique()
grouped_df = student_df.groupby("parental level of education")
for var in ["math score", "reading score", "writing score"]:
    print("------------------------------------------------")
    print("Comparisons for variable: {}".format(var))
    print("------------------------------------------------")
    for i in range(0, len(parent)):
        for j in range(i+1, len(parent)):
            print(
                "t-test between groups {0} and {1}:".format(parent[i], parent[j]))
            print(stats.ttest_ind(
                student_df[student_df["parental level of education"]
                           == parent[i]][var],
                student_df[student_df["parental level of education"]
                           == parent[j]][var]
            ))

------------------------------------------------
Comparisons for variable: math score
------------------------------------------------
t-test between groups bachelor's degree and some college:
Ttest_indResult(statistic=1.3702352829617477, pvalue=0.17151224218471559)
t-test between groups bachelor's degree and master's degree:
Ttest_indResult(statistic=-0.14868196568849312, pvalue=0.8819757824867702)
t-test between groups bachelor's degree and associate's degree:
Ttest_indResult(statistic=0.8786641889793243, pvalue=0.38020757818036177)
t-test between groups bachelor's degree and high school:
Ttest_indResult(statistic=4.236143622484822, pvalue=2.9965345853843376e-05)
t-test between groups bachelor's degree and some high school:
Ttest_indResult(statistic=3.196718602676573, pvalue=0.0015409192922812146)
t-test between groups some college and master's degree:
Ttest_indResult(statistic=-1.2356334215824571, pvalue=0.21761956810347416)
t-test between groups some college and associate's degree:

There is definietly a relationship betweent the parent's level of education and the student's test scores.  The higher the level of education for the parets, the higher the average test score across all three subjects.  The T-tests indicate that as the distance in education levels increases (ie. Master's degree versus some high school), the difference in means is significant.  

### 2) Are there any differences between the lunch types with respect to their performances in exams? If there are, how do you explain this?

In [10]:
#Groupby lunch type
lunch=student_df.groupby(['lunch']).mean()
lunch

,math score,reading score,writing score
lunch,,,
free/reduced,58.921127,64.653521,63.022535
standard,70.034109,71.654264,70.823256


In [11]:
#Perform t test to determine if differences in score are significant

stats.ttest_ind(student_df[student_df.lunch =='free/reduced'][['math score', 'writing score', 'reading score']],
                student_df[student_df.lunch =='standard'][['math score', 'writing score', 'reading score']]
)

Ttest_indResult(statistic=array([-11.83718047,  -8.0097842 ,  -7.45105647]), pvalue=array([2.41319560e-30, 3.18618958e-15, 2.00279665e-13]))

The type of lunch served appears to have a relationship with the student's tests scores.  The t-test indicates that there is a statistically  significant differnce between student who ate free lunches and those who ate the standard lunch.  While on the surface this may not seem to have much correlation, the measure of free versus standard lunch is really a measure of socio-economic status and whether or not those students who receive 'free lunches' are sufficiently nurioused throughout the day.  Many studies have found that hungry students can't focus well on their studies, thus the relationship between lunch type and grades.

### 3) Does the test preparation course seem to have an effect on the exam performances?

In [33]:
#Run correlation between test prep and scores
student_df.groupby(['test preparation course']).mean()

,math score,reading score,writing score
test preparation course,,,
completed,69.695531,73.893855,74.418994
none,64.077882,66.534268,64.504673


In [34]:
#Perform statistical test on differences

stats.ttest_ind(student_df[student_df['test preparation course'] == 'completed'][['math score', 'reading score','writing score']],
                student_df[student_df['test preparation course'] == 'none'][['math score', 'reading score', 'writing score']]
               )

Ttest_indResult(statistic=array([ 5.70461642,  7.87166354, 10.40917344]), pvalue=array([1.53591346e-08, 9.08178334e-15, 3.68529174e-24]))

### 4) Which 2 exam scores are most correlated with each other?

In [36]:
#Correlation matrix for test scores
student_df[['math score', 'reading score', 'writing score']].corr()

,math score,reading score,writing score
math score,1.000000,0.817580,0.802642
reading score,0.817580,1.000000,0.954598
writing score,0.802642,0.954598,1.000000
